# Decision Tree Regressor with Pyspark low-level API

## 1. Preparation

### Importing necessary packages

In [1]:
from pyspark import SparkContext, SparkConf, RDD
from pyspark.statcounter import StatCounter
from datetime import datetime

(Optional) Set the memory usage configurations for Pyspark session:

In [2]:
#Set the config for spark to boost performance
config = SparkConf()\
            .set("spark.driver.memory", "4g")\
            .set("spark.executor.memory", "8g")

Let's initialize a Spark session:

In [3]:
#pyspark init
sc = SparkContext(appName='taxi_duration_lowlevel', conf=config).getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/04/05 15:36:10 WARN Utils: Your hostname, DESKTOP-0H87CFM resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/05 15:36:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/05 15:36:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## 2. Data preprocessing

Read input files:

In [4]:
def read_csv(filepath: str):
    """Read csv file into a rdd of values and a list of feature columns separately."""
    #Read the data into rdd
    raw_rdd = sc.textFile(filepath)

    #Remove the csv header row
    header = raw_rdd.first()

    #Strip the header row
    rdd_no_header = raw_rdd.filter(lambda row: row != header)

    return rdd_no_header, header.split(',')


raw_train_rdd, train_header = read_csv('train.csv')
raw_test_rdd, test_header = read_csv('test.csv')

Filter usable columns:

In [5]:
def extract_column(row: str, header: list[str], excluding_features: list[str]):
    """Extract usable feature columns with given excluding filter."""

    vector = []

    #Split the row with delimiter `,`
    values = row.split(',')
    
    #Filter values
    for feature, value in zip(header, values):
        if feature not in excluding_features:
            vector.append((feature, value))
                
    return dict(vector)

Preprocess features into usable form:

In [6]:
def preprocess_data(row: dict, label_col = None):
    """- Flatten `pickup_datetime` into separate elements (day, month, year,...).
       - Encode `store_and_fwd_flag` to binary values.
       - Cast strings to numeric values."""

    dict_row = {}

    #Flatten datetime strings into component elements
    row['pickup_datetime'] = row['pickup_datetime'].replace('-', ':').replace(' ', ':').split(':')

    #Encode binary values feature
    row['store_and_fwd_flag'] = 1 if row['store_and_fwd_flag'] == 'Y' else 0

    #Cast strings to float
    features = [float(value) for value in [
                    row['vendor_id'],
                    *row['pickup_datetime'],
                    row['passenger_count'],
                    row['pickup_longitude'],
                    row['pickup_latitude'],
                    row['dropoff_longitude'],
                    row['dropoff_latitude'],
                    row['store_and_fwd_flag']
                ]
        ]
    
    #Return a dict of { 'features': vector_of_values, 'label': label (if any) }
    dict_row['features'] = features
    dict_row['label'] = float(row[label_col]) if label_col else None

    return dict_row

train_rdd = raw_train_rdd.map(lambda row: extract_column(row, train_header, ['id', 'dropoff_datetime']))\
                         .map(lambda row: preprocess_data(row, 'trip_duration'))

    
test_rdd = raw_test_rdd.map(lambda row: extract_column(row, test_header, ['id']))\
                       .map(lambda row: preprocess_data(row, None))

## 3. Model training and testing 

### Ultility functions

`partition_feature_grouping` for flattening each row of dataset into a tuple of (feature_value, row_label):

In [7]:
def partition_feature_grouping(partition, usable_features: list):
    """Read each row and convert into a list of (feature_value, row_label), then flatten the results."""
    feature_stats = {}

    for row in partition:
        for feature in usable_features:
            # Store feature-wise values in a dictionary
            if feature not in feature_stats:
                feature_stats[feature] = []

            feature_stats[feature].append((row['features'][feature], row['label']))

    results = []
    
    for feature, values in feature_stats.items():
        results.append((feature, values))

    return iter(results)

`find_split_feature` for finding splitting point with maximum variance reduction on the domain of a feature:

In [8]:
def find_split_feature(values: list[list[float ] ], parent_info: StatCounter):
    """Find the splitting point (best variance reduction) of a continuous-values feature using left-right separation, return the mean of 2 values at the border."""
    
    #Sort all values in ascending order
    sorted_values = sorted(values)

    #Get the number of values
    parent_count = parent_info.count()

    #Get the E(X^2) of the whole population based on E(X^2) = Var(X) + E^2(X)
    parent_pow_sum = parent_info.variance() + parent_info.mean() ** 2
    

    left_sum, left_pow_sum, left_count = 0, 0, 0
    best_split = (-float("inf"), None) 


    for i in range(0, parent_count - 1):
        feature_value, label = sorted_values[i]

        #Accumulative sum of y, sum of y^2 and number of values on the left of the currently inspecting splitting point
        left_sum += label
        left_pow_sum += label ** 2
        left_count += 1

        #For filtering a sequence of similar values, only consider a valid split between 2 different values
        if feature_value == sorted_values[i+1][0]:
            continue

        #Infer the accumulative results on the right side from the parent and the left information
        right_count = parent_count - left_count
        right_sum = parent_info.sum() - left_sum
        right_pow_sum = parent_pow_sum - left_pow_sum

        #Compute variance of the left subset, right subset and the variance reduction using this splitting point
        left_var = left_pow_sum / left_count - (left_sum / left_count)**2 if left_count > 0 else 0
        
        right_var = right_pow_sum / right_count - (right_sum / right_count)**2 if right_count > 0 else 0

        var_reduction = parent_info.variance() - left_var * (left_count / parent_count) - right_var * (right_count / parent_count)     

        #Update the best splitting point information         
        if var_reduction > best_split[0]:
            best_split = ( var_reduction, (feature_value + sorted_values[i+1][0]) / 2 )

    return best_split

Sub-function `splitter` for splitting the rows of the parent dataset according to the splitting point and operands:

In [9]:
def splitter(iterator, split_feature: int, split_point: float, operand):
    ret = []
    for row in iterator:
        if operand(row['features'][split_feature], split_point):
            ret.append(row)

    return iter(ret)

Main class `DecisionTreeRegressor` for building and executing Decision Tree Regressor Algorithm:

In [10]:
class DecisionTreeRegressor:
    def __init__(self, max_depth = 1):
        #Initialize the estimator with given depth (if any)
        self.max_depth = max_depth
        self.rules = None
        pass

    def set_maxDepth(self, depth):
        """Set current maximum depth for the estimator."""
        self.max_depth = depth

    def fit(self, train_rdd: RDD):
        """Execute Decision Tree Algorithm recursively on a given rdd based on variance reduction and the current maximum depth."""
        self.num_features = len(train_rdd.first()['features'])
        self.usable_features = [i for i in range(self.num_features)]
        sample_size = train_rdd.count()
        
        return self.__build_rule_tree_recursive(train_rdd, self.usable_features, sample_size)

    def transform(self, rdd: RDD):
        
        pass

    def display_rule_tree(self, model: dict):
        """Recursively display the rules of a Decision Tree model."""
        self.__display_tree_recusive(model)



    def __display_tree_recusive(self, rules: dict, indent = 0):
        #Stopping condition
        if not rules:
            return

        if len(rules) == 1:
            #Is a leaf condition
            print(f"{indent * ' '}Predict: {rules['prediction']}")
            return

        #Print the splitting point information and call recursion of the left and right child

        print(f"{indent * ' '}If  features[{rules['split_feature']}] <= {rules['split_point']}")
        self.__display_tree_recusive(rules['left'], indent + 5)
        print(f"{indent * ' '}Else  features[{rules['split_feature']}] > {rules['split_point']}")
        self.__display_tree_recusive(rules['right'], indent + 5)         

    def __find_best_split(self, rdd: RDD, usable_features: list):         
        """Find the best splitting point with maximum variance reduction of the input dataset."""

        #Compute the dataset statistics and store into a StatCounter object
        parent_info = rdd.mapPartitions(lambda partition: [StatCounter([row['label'] for row in partition])], True)\
                        .reduce(lambda stat1, stat2: stat1.mergeStats(stat2))
        
        #Convert each row into a list of (row_feature_value, row_label) and flatten the results
        processed_rdd = rdd.mapPartitions(lambda partition: partition_feature_grouping(partition, usable_features), True)\
                      .reduceByKey(lambda l1, l2: l1 + l2).cache()
        
        #Find the best splitting point for each features
        best_split_per_feature = processed_rdd.mapValues(lambda values: find_split_feature(values, parent_info))\
                                  
        #Find the best splitting point for the dataset
        best_split = max(best_split_per_feature.collect(),key=lambda x: (x[1][0], -x[0]))

        return best_split[0], best_split[1][1]


    def __build_rule_tree_recursive(self, parent: RDD, usable_features: list, sample_size, depth = 0):
        """Recursively build the decision tree by finding the splitting point with maximum variance reduction and split the dataset with this point, up to the maximum depth."""
        
        #Stopping condition
        if sample_size == 0:
            return None
        
        #Return the mean of un-splitted subset as the prediction if reached the depth limit
        if depth == self.max_depth:
            mean = parent.mapPartitions(lambda partition: [StatCounter([row['label'] for row in partition])], True)\
                        .reduce(lambda stat1, stat2: stat1.mergeStats(stat2)).mean()

            return {'prediction' : mean}

        #Find the best splitting point for the input dataset
        split_feature, split_point = self.__find_best_split(parent, usable_features)
        
        #Return the mean of un-splitted subset as the prediction if no splitting point is valid but has not reached the depth limit yet
        if split_point == None:
            mean = parent.mapPartitions(lambda partition: [StatCounter([row['label'] for row in partition])], True)\
                        .reduce(lambda stat1, stat2: stat1.mergeStats(stat2)).mean()
            
            return {'prediction' : mean}

        #Split the dataset into left and right subsets
        left_rdd = parent.mapPartitions(lambda iterator: splitter(iterator, split_feature, split_point, lambda a,b: a <= b), True).cache()
        right_rdd = parent.mapPartitions(lambda iterator: splitter(iterator, split_feature, split_point, lambda a,b: a >= b), True).cache()
        
        #Un-cache the parent dataset (excluding the input one)
        if depth > 0:
            parent.unpersist()

        #Compute the size of the left subset
        left_sample_size = left_rdd.count()
     
        #Build the dict of information with recursion call
        return {
            'split_feature' : split_feature,
            'split_point' : split_point,
            'left' : self.__build_rule_tree_recursive(left_rdd, usable_features, left_sample_size, depth + 1),
            'right' : self.__build_rule_tree_recursive(right_rdd, usable_features, sample_size - left_sample_size, depth + 1)
        }

In [11]:
estimator = DecisionTreeRegressor(5)
model = estimator.fit(train_rdd)
estimator.display_rule_tree(model)

If  features[8] <= -73.88602066040039
     If  features[10] <= -73.91103744506836
          If  features[11] <= 40.70149040222168
               If  features[9] <= 40.725751876831055
                    If  features[11] <= 40.701486587524414
                         Predict: 1019.1713947990531
                    Else  features[11] > 40.701486587524414
                         Predict: 28934.666666666668
               Else  features[9] > 40.725751876831055
                    If  features[9] <= 40.72575569152832
                         Predict: 22868.0
                    Else  features[9] > 40.72575569152832
                         Predict: 1901.4971375584564
          Else  features[11] > 40.70149040222168
               If  features[0] <= 1.5
                    If  features[8] <= -73.92168045043945
                         Predict: 712.2167951813674
                    Else  features[8] > -73.92168045043945
                         Predict: 2017.9384793964005
               Else